In [18]:
import numpy as np
import pandas as pd
from scipy import stats

In [19]:
def gerber(c, arr1, arr2):#c is weight
    t1 = c*np.std(arr1)
    t2 = c*np.std(arr2)
    concordant = 0
    discordant = 0
    for i in arr1:
        for j in arr2:
            if i >= t1 and j >= t2:
                concordant += 1
            elif i <= -t1 and j <= -t2:
                concordant += 1
            elif i >= t1 and j <= -t2:
                discordant -= 1
            elif i <= -t1 and j >= t2:
                discordant -= 1
    return (concordant + discordant)/(concordant - discordant)

In [20]:
csv_path = '.\\NN4.csv'

c = 0.5

results = pd.read_csv(csv_path,index_col=0)
df = pd.DataFrame(columns=['pearson','spearman','gerber'])
gb = results.groupby(by = 'date')
pearson = gb.apply(lambda df: np.corrcoef(df['ret'], df['pred_ret'])[0][1])
spearman = gb.apply(lambda df: stats.spearmanr(df['ret'], df['pred_ret'])[0])
gerber_cor = gb.apply(lambda df: gerber(c, df['ret'], df['pred_ret']))
print(gerber)
df['pearson'] = pearson
df['spearman'] = spearman
df['gerber'] = gerber_cor
df.to_csv('.\\measures.csv')
print(df)


<function gerber at 0x000002B8138A5820>
             pearson  spearman    gerber
date                                    
2010-02-01  0.013792  0.067757  0.604309
2010-03-01  0.049657  0.097754  0.487864
2010-04-01  0.014452  0.031213 -0.614457
2010-04-30  0.048020  0.073695  0.386114
2010-06-01  0.131904  0.151445 -0.647024
...              ...       ...       ...
2021-07-01  0.089702  0.090725 -0.017525
2021-07-30 -0.028056 -0.021639  0.392218
2021-09-01  0.060743  0.024301 -0.269868
2021-09-30 -0.098785 -0.077467 -0.283931
2021-11-01 -0.038375 -0.039894  0.490467

[142 rows x 3 columns]


In [21]:
""" 接下来我们验证我们的gerber method确实按照是公众号文章里面的计算方式计算的
我们generate两组长度为10000的N(0,1)变量，按照文章中的计算方式，这两组的变量的gerber correlation在parameter的任何取值情况下都应该是0
"""


' 接下来我们验证我们的gerber method确实按照是公众号文章里面的计算方式计算的\n我们generate两组长度为10000的N(0,1)变量，按照文章中的计算方式，这两组的变量的gerber correlation在parameter的任何取值情况下都应该是0\n'

In [29]:
def gerber(c, arr1, arr2):#c is weight
    t1 = c*np.std(arr1)
    t2 = c*np.std(arr2)
    concordant = 0
    discordant = 0
    print(t1,t2)
    for i in arr1:
        for j in arr2:
            if i >= t1 and j >= t2:
                concordant += 1
            elif i <= -t1 and j <= -t2:
                concordant += 1
            elif i >= t1 and j <= -t2:
                discordant -= 1
            elif i <= -t1 and j >= t2:
                discordant -= 1
    print(concordant, discordant)
    return (concordant + discordant)/(concordant - discordant)

In [22]:

x = np.random.normal(0,1,(2,10000))

gerber0 = gerber(0, x[0], x[1])
gerber1 = gerber(0.2, x[0], x[1])
gerber2 = gerber(0.4, x[0], x[1])
gerber3 = gerber(0.5, x[0], x[1])
gerber4 = gerber(0.7, x[0], x[1])

print(gerber0,gerber1,gerber2,gerber3,gerber4)

0.0 6.614576780297087e-05 8.022661952318922e-05 0.00012850390054689395 9.460697469791817e-05


In [34]:
""" 
我们接着在(2,2)处generate两组长度为10000的N(0,1)变量，按照文章中的计算方式，这两组的变量的gerber correlation在0处取得最小值，当parameter大于0时，discordant的数量会显著下降。（discordant的threshold离(2,2)的距离越来越远）
"""


' \n我们接着在(2,2)处generate两组长度为10000的N(0,1)变量，按照文章中的计算方式，这两组的变量的gerber correlation在parameter小于2时也都应该是1: 2和-x之间有四个标准差的距离\n'

In [35]:
x = np.random.normal(0,1,(2,10000))
x += np.array([[2],[2]])

gerber0 = gerber(0, x[0], x[1])
gerber1 = gerber(1, x[0], x[1])
gerber2 = gerber(1.9, x[0], x[1])
gerber3 = gerber(2.1, x[0], x[1])


print(gerber0,gerber1,gerber2,gerber3)

0.0 0.0
95582150 -4417850
0.9909708349054103 0.9967470527458253
71208440 -227864
1.8828445863202794 1.893819400217068
30063189 -5473
2.0810387533013617 2.093168810766233
22406022 -4733
0.911643 0.9936204986193015 0.9996359665089188 0.9995776135163674


In [36]:
""" 
为了得到一组gerber corr不为正负1的vector pair, 我们在(1,1)处generate两组长度为10000的N(0,1)变量，按照文章中的计算方式，这两组的变量的gerber correlation在parameter大于1时也都应该靠近1：2和-x, where x>1之间有大于2个标准差的距离，这会导致没有discordant

"""


' \n为了得到一组gerber corr不为正负1的vector pair, 我们在(1,1)处generate两组长度为10000的N(0,1)变量，按照文章中的计算方式，这两组的变量的gerber correlation在parameter大于1时也都应该靠近1：2和-x, where x>1之间有大于2个标准差的距离，这会导致没有discordant\n\n'

In [31]:
x = np.random.normal(0,1,(2,10000))
x += np.array([[1],[1]])

gerber0 = gerber(0, x[0], x[1])
gerber1 = gerber(1, x[0], x[1])
gerber2 = gerber(3, x[0], x[1])


print(gerber0,gerber1,gerber2)

0.0 0.0
73419590 -26580410
1.0143274394741246 1.0052128063005241
24882564 -2391810
3.042982318422374 3.015638418901572
50370 0
0.4683918 0.8246111899763492 1.0
